# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 67 new papers today
          27 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/26 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2212.06838


extracting tarball to tmp_2212.06838...

 done.
Retrieving document from  https://arxiv.org/e-print/2212.06845


extracting tarball to tmp_2212.06845...

...

 done.
Retrieving document from  https://arxiv.org/e-print/2212.07416


/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Found documentclass in tmp_2212.07399/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'cloudparams.tex' from 'tmp_2212.07399/cloudparams.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'cloudparams.tex' from 'tmp_2212.07399/2M2139+SIMP0136 Retrieval/cloudparams.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.15/x6

extracting tarball to tmp_2212.07416...

 done.


/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Found documentclass in tmp_2212.07416/CBD-simulation.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.07332-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.07332) | **Two temperate Earth-mass planets orbiting the nearby star GJ1002**  |
|| A. Suárez Mascareño, et al. -- incl., <mark>J. Lillo-Box</mark>, <mark>J. I. González Hernández</mark>, <mark>Th. Henning</mark> |
|*Appeared on*| *2022-12-15*|
|*Comments*| *23 pages, 20 figures, 3 tables*|
|**Abstract**| We report the discovery and characterisation of two Earth-mass planets orbiting in the habitable zone of the nearby M-dwarf GJ~1002 based on the analysis of the radial-velocity (RV) time series from the ESPRESSO and CARMENES spectrographs. The host star is the quiet M5.5~V star GJ~1002 (relatively faint in the optical, $V \sim 13.8$ mag, but brighter in the infrared, $J \sim 8.3$ mag), located at 4.84 pc from the Sun. We analyse 139 spectroscopic observations taken between 2017 and 2021. We performed a joint analysis of the time series of the RV and full-width half maximum (FWHM) of the cross-correlation function (CCF) to model the planetary and stellar signals present in the data, applying Gaussian process regression to deal with the stellar activity. We detect the signal of two planets orbiting GJ~1002. GJ~1002~b is a planet with a minimum mass $m_p \sin i $ of 1.08 $\pm$ 0.13 M$_{\oplus}$ with an orbital period of 10.3465 $\pm$ 0.0027 days at a distance of 0.0457 $\pm$ 0.0013 au from its parent star, receiving an estimated stellar flux of 0.67 $F_{\oplus}$. GJ~1002 c is a planet with a minimum mass $m_p \sin i $ of 1.36 $\pm$ 0.17 M$_{\oplus}$ with an orbital period of 20.202 $\pm$ 0.013 days at a distance of 0.0738 $\pm$ 0.0021 au from its parent star, receiving an estimated stellar flux of 0.257 $F_{\oplus}$. We also detect the rotation signature of the star, with a period of 126 $\pm$ 15 days. GJ~1002 is one of the few known nearby systems with planets that could potentially host habitable environments. The closeness of the host star to the Sun makes the angular sizes of the orbits of both planets ($\sim$ 9.7 mas and $\sim$ 15.7 mas, respectively) large enough for their atmosphere to be studied via high-contrast high-resolution spectroscopy with instruments such as the future spectrograph ANDES for the ELT or the LIFE mission. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.07040-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.07040) | **Overview of the Observing System and Initial Scientific Accomplishments  of the East Asian VLBI Network (EAVN)**  |
|| Kazunori Akiyama, et al. -- incl., <mark>Lang Cui</mark>, <mark>Wen Guo</mark>, <mark>Busaba H. Kramer</mark>, <mark>Jee-Won Lee</mark>, <mark>Jeong Ae Lee</mark>, <mark>Sang-Sung Lee</mark>, <mark>Sang Won Lee</mark>, <mark>Bin Li</mark>, <mark>Guanghui Li</mark>, <mark>Xiaofei Li</mark>, <mark>Zhixuan Li</mark>, <mark>Qinghui Liu</mark>, <mark>Xiang Liu</mark> |
|*Appeared on*| *2022-12-15*|
|*Comments*| *27 pages, appeared in Galaxies special issue 'Challenges in Understanding Black Hole Powered Jets with VLBI' as an invited review*|
|**Abstract**| The East Asian VLBI Network (EAVN) is an international VLBI facility in East Asia and is operated under mutual collaboration between East Asian countries, as well as part of Southeast Asian and European countries. EAVN currently consists of 16 radio telescopes and three correlators located in China, Japan, and Korea, and is operated mainly at three frequency bands, 6.7, 22, and 43 GHz with the longest baseline length of 5078 km, resulting in the highest angular resolution of 0.28 milliarcseconds at 43 GHz. One of distinct capabilities of EAVN is multi-frequency simultaneous data reception at nine telescopes, which enable us to employ the frequency phase transfer technique to obtain better sensitivity at higher observing frequencies. EAVN started its open-use program in the second half of 2018, providing a total observing time of more than 1100 hours in a year. EAVN fills geographical gap in global VLBI array, resulting in enabling us to conduct contiguous high-resolution VLBI observations. EAVN has produced various scientific accomplishments especially in observations toward active galactic nuclei, evolved stars, and star-forming regions. These activities motivate us to initiate launch of the 'Global VLBI Alliance' to provide an opportunity of VLBI observation with the longest baselines on the earth. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.06838-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.06838) | **Cosmology with cosmic web environments II. Redshift-space auto and cross  power spectra**  |
|| Tony Bonnaire, <mark>Joseph Kuruvilla</mark>, Nabila Aghanim, Aurélien Decelle |
|*Appeared on*| *2022-12-15*|
|*Comments*| *10 pages, 6 figures, 3 tables. Comments are welcome*|
|**Abstract**| Degeneracies among parameters of the cosmological model are known to drastically limit the information contained in the matter distribution. In the first paper of this series, we shown that the cosmic web environments; namely the voids, walls, filaments and nodes; can be used as a leverage to improve the real-space constraints on a set of six cosmological parameters, including the summed neutrino mass. Following-upon these results, we propose to study the achievable constraints of environment-dependent power spectra in redshift space where the velocities add up information to the standard two-point statistics by breaking the isotropy of the matter density field. A Fisher analysis based on a set of thousands of Quijote simulations allows us to conclude that the combination of power spectra computed in the several cosmic web environments is able to break some degeneracies. Compared to the matter monopole and quadrupole information alone, the combination of environment-dependent spectra tightens down the constraints on key parameters like the matter density or the summed neutrino mass by up to a factor of $5.5$. Additionally, while the information contained in the matter statistic quickly saturates at mildly non-linear scales in redshift space, the combination of power spectra in the environments appears as a goldmine of information able to improve the constraints at all the studied scales from $0.1$ to $0.5$ $h$/Mpc and suggests that further improvements are reachable at even finer scales. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.07366-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.07366) | **The first large catalogue of spectroscopic redshifts in Webb's First  Deep Field, SMACS J0723.3$-$7327**  |
|| Gaël Noirot, et al. -- incl., <mark>Jasleen Matharu</mark> |
|*Appeared on*| *2022-12-15*|
|*Comments*| *19 pages and 13 figures (incl. appendices), submitted to MNRAS*|
|**Abstract**| We present a spectroscopic redshift catalogue of the SMACS J0723.3$-$7327 field ("Webb's First Deep Field") obtained from JWST/NIRISS grism spectroscopy and supplemented with JWST/NIRSpec and VLT/MUSE redshifts. The catalogue contains a total of 190 sources with secure spectroscopic redshifts, including 156 NIRISS grism redshifts, 123 of which are for sources whose redshifts were previously unknown. These new grism redshifts are secured with two or more spectroscopic features (64 sources), or with a single spectral feature whose identity is secured from the object's nine-band photometric redshift (59 sources). These are complemented with 17 NIRSpec and 48 MUSE redshifts, including six new NIRSpec redshifts identified in this work. In addition to the $z_{\rm cl}=0.39$ cluster galaxy redshifts (for which we provide $\sim$40 new NIRISS absorption-line redshifts), we also find three prominent galaxy overdensities at higher redshifts - at $z=1.1$, $z=1.4$, and $z=2.0$ - that were until now not seen in the JWST/NIRSpec and VLT/MUSE data. The paper describes the characteristics of our spectroscopic redshift sample and the methodology we have employed to obtain it. Our redshift catalogue is made available to the community at https://niriss.github.io/smacs0723. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.07373-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.07373) | **The ESO-VLT MIKiS survey reloaded: exploring the internal kinematics of  NGC 6440**  |
|| Silvia Leanza, et al. -- incl., <mark>Livia Origlia</mark> |
|*Appeared on*| *2022-12-15*|
|*Comments*| *20 pages, 11 figures, 4 tables, accepted for publication in ApJ*|
|**Abstract**| In the context of the ESO-VLT Multi-Instrument Kinematic Survey (MIKiS) of Galactic globular clusters, here we present the line-of-sight velocity dispersion profile of NGC 6440, a massive globular cluster located in the Galactic bulge. By combining the data acquired with four different spectrographs, we obtained the radial velocity of a sample of $\sim 1800$ individual stars distributed over the entire cluster extension, from $\sim$0.1$"$ to 778$"$ from the center. Using a properly selected sample of member stars with the most reliable radial velocity measures, we derived the velocity dispersion profile up to 250$"$ from the center. The profile is well described by the same King model that best fits the projected star density distribution, with a constant inner plateau (at ${\sigma}_0 \sim $ 12 km s$^{-1}$) and no evidence of a central cusp or other significant deviations. Our data allowed to study the presence of rotation only in the innermost regions of the cluster (r < 5$"$), revealing a well-defined pattern of ordered rotation with a position angle of the rotation axis of $\sim$132 $\pm$ 2{\deg} and an amplitude of $\sim$3 km s$^{-1}$ (corresponding to Vrot/${\sigma}_0 \sim$ 0.3). Also, a flattening of the system qualitatively consistent with the rotation signal has been detected in the central region. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.07386-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.07386) | **Quantum Tunneling of Ultralight Dark Matter Out of Satellite Galaxies**  |
|| <mark>Mark P. Hertzberg</mark>, Abraham Loeb |
|*Appeared on*| *2022-12-15*|
|*Comments*| *6 pages, 2 figures, in double column format*|
|**Abstract**| The idea of ultralight scalar (axion) dark matter is theoretically appealing and may resolve some small-scale problems of cold dark matter; so it deserves careful attention. In this work we carefully analyze tunneling of the scalar field in dwarf satellites due to the tidal gravitational force from the host halo. The tidal force is far from spherically symmetric; causing tunneling along the axis from the halo center to the dwarf, while confining in the orthogonal plane. We decompose the wave function into a spherical term plus higher harmonics, integrate out angles, and then numerically solve a residual radial Schr\"odinger-Poisson system. By demanding that the core of the Fornax dwarf halo can survive for at least the age of the universe places a bound on the dark matter particle mass $2\times 10^{-22}\,\mbox{eV}\lesssim m\lesssim 6\times 10^{-22}\,$eV. Interestingly, we show that if another very low density halo is seen, then it rules out the ultralight scalar as core proposal completely. Furthermore, the non-condensed particles likely impose an even sharper lower bound. We also determine how the residual satellites could be distributed as a function of radius. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.07399-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.07399) | **Patchy Forsterite Clouds in the Atmospheres of Two Highly Variable  Exoplanet Analogs**  |
|| Johanna M. Vos, et al. -- incl., <mark>Ben Burningham</mark> |
|*Appeared on*| *2022-12-15*|
|*Comments*| *Accepted for publication in ApJ*|
|**Abstract**| We present an atmospheric retrieval analysis of a pair of highly variable, $\sim200~$Myr old, early-T type planetary-mass exoplanet analogs SIMP J01365662+0933473 and 2MASS J21392676+0220226 using the Brewster retrieval framework. Our analysis, which makes use of archival $1-15~\mu$m spectra, finds almost identical atmospheres for both objects. For both targets, we find that the data is best described by a patchy, high-altitude forsterite (Mg$_2$SiO$_4$) cloud above a deeper, optically thick iron (Fe) cloud. Our model constrains the cloud properties well, including the cloud locations and cloud particle sizes. We find that the patchy forsterite slab cloud inferred from our retrieval may be responsible for the spectral behavior of the observed variability. Our retrieved cloud structure is consistent with the atmospheric structure previously inferred from spectroscopic variability measurements, but clarifies this picture significantly. We find consistent C/O ratios for both objects which supports their formation within the same molecular cloud in the Carina-Near Moving Group. Finally, we note some differences in the constrained abundances of H$_2$O and CO which may be caused by data quality and/or astrophysical processes such as auroral activity and their differing rotation rates. The results presented in this work provide a promising preview of the detail with which we will characterize extrasolar atmospheres with JWST, which will yield higher quality spectra across a wider wavelength range. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.07416-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.07416) | **Hydrodynamical Simulations of Circumbinary Accretion: Balance between  Heating and Cooling**  |
|| Hai-Yang Wang, Xue-Ning Bai, Dong Lai, <mark>Douglas N. C. Lin</mark> |
|*Appeared on*| *2022-12-15*|
|*Comments*| *19 pages, 19 figures, submitted to MNARS*|
|**Abstract**| Hydrodynamical interaction in circumbinary discs (CBDs) plays a crucial role in various astrophysical systems, ranging from young stellar binaries to supermassive black hole binaries in galactic centers. Most previous simulations of binary-disc systems have adopted locally isothermal equation of state. In this study, we use the grid-based code $\texttt{Athena++}$ to conduct a suite of two-dimensional viscous hydrodynamical simulations of circumbinary accretion on a cartesian grid, resolving the central cavity of the binary. The gas thermodynamics is treated by thermal relaxation towards an equilibrium temperature (based on the constant$-\beta$ cooling ansatz, where $\beta$ is the cooling time in units of the local Keplerian time). Focusing on equal mass, circular binaries in CBDs with (equilibrium) disc aspect ratio $H/R=0.1$, we find that the cooling of the disc gas significantly influences the binary orbital evolution, accretion variability, and CBD morphology, and the effect depends sensitively on the disc viscosity prescriptions. When adopting a constant kinematic viscosity, a finite cooling time ($\beta \gtrsim 0.1$) leads to binary inspiral as opposed to outspiral and the CBD cavity becomes more symmetric. When adopting a dynamically varying $\alpha-$viscosity, binary inspiral only occurs within a narrow range of cooling time (corresponding to $\beta$ around 0.5). |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2212.07332.md
    + _build/html/tmp_2212.07332/./APPEND_ACT/ACT_APPEND_DATA.png
    + _build/html/tmp_2212.07332/./IMG_DATA_OBS/RV_DATA_2PC.png
    + _build/html/tmp_2212.07332/./IMG_DATA_OBS/SNO_DATA.png
    + _build/html/tmp_2212.07332/./ANALYSIS/RV/RV_DATA_MODEL_0P.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$</div>



<div id="title">

# The data used in this paper is available in electronic form at the CDS via anonymous ftp to cdsarc.u-strasbg.fr (130.79.128.5) or via http://cdsweb.u-strasbg.fr/cgi-bin/qcat?J/A+A/

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2212.07332-b31b1b.svg)](https://arxiv.org/abs/2212.07332)<mark>Appeared on: 2022-12-15</mark> - _23 pages, 20 figures, 3 tables_

</div>
<div id="authors">

A. Suárez Mascareño, et al. -- incl., <mark><mark>J. Lillo-Box</mark></mark>, <mark><mark>J. I. González Hernández</mark></mark>, <mark><mark>Th. Henning</mark></mark>

</div>
<div id="abstract">

**Abstract:** We report the discovery and characterisation of two Earth-mass planets orbiting in the habitable zone of the nearby M-dwarf GJ 1002 based on the analysis of the radial-velocity (RV) time series from the ESPRESSO and CARMENES spectrographs. The host star is the quiet M5.5 V star GJ 1002  (relatively faint in the optical,$V \sim 13.8$mag, but brighter in the infrared,$J \sim 8.3$mag), located at 4.84 pc from the Sun. We analyse 139 spectroscopic observations taken between 2017 and 2021. We performed a joint analysis of the time series of the RV and full-width half maximum (FWHM) of the cross-correlation function (CCF) to model the planetary and stellar signals present in the data, applying Gaussian process regression to deal with the stellar activity. We detect the signal of two planets orbiting GJ 1002. GJ 1002 b is a planet with a minimum mass$m_p \sin i $of 1.08$\pm$0.13 M$_{\oplus}$with an orbital period of 10.3465$\pm$0.0027 days at a distance of 0.0457$\pm$0.0013 au from its parent star, receiving an estimated stellar flux of 0.67$F_{\oplus}$. GJ 1002 c is a planet with a minimum mass$m_p \sin i $of 1.36$\pm$0.17 M$_{\oplus}$with an orbital period of 20.202$\pm$0.013 days at a distance of 0.0738$\pm$0.0021 au from its parent star, receiving an estimated stellar flux of 0.257$F_{\oplus}$. We also detect the rotation signature of the star, with a period of 126$\pm$15 days. We find that there is a correlation between the temperature of certain optical elements in the spectrographs and changes in the instrumental profile that can affect the scientific data, showing a seasonal behaviour that creates spurious signals at periods longer than$\sim$200 days. GJ 1002 is one of the few known nearby systems with planets that could potentially host habitable environments. The closeness of the host star to the Sun makes the angular sizes of the orbits of both planets ($\sim$9.7 mas and$\sim$15.7 mas, respectively) large enough for their atmosphere to be studied via high-contrast high-resolution spectroscopy with instruments such as the future spectrograph ANDES for the ELT or the LIFE mission.

</div>

<div id="div_fig1">

<img src="tmp_2212.07332/./APPEND_ACT/ACT_APPEND_DATA.png" alt="Fig19" width="100%"/>

**Figure 19. -** Activity indicators (left panels) and their GLS periodograms (right panels). All spectroscopic indictors have been detrended against the temperature of the \'Echelle grating. The green shaded regions in the GLS periodograms show the periods of the planets GJ 1002 b and c. The widths of the region around the periods have been exaggerated for better visualisation. The red shaded regions shows the 95\% confidence interval around the measured rotation period. In all cases, the median value of each dataset has been subtracted. (*activity_ind*)

</div>
<div id="div_fig2">

<img src="tmp_2212.07332/./IMG_DATA_OBS/RV_DATA_2PC.png" alt="Fig6.1" width="50%"/><img src="tmp_2212.07332/./IMG_DATA_OBS/SNO_DATA.png" alt="Fig6.2" width="50%"/>

**Figure 6. -** Scientific data used in the subsequent analysis. \textit{Top panel}: RV data of GJ 1002. The purple filled circles show the CARMENES data. The teal symbols show the ESPRESSO data. \textit{Middle panel}: FWHM time series of GJ 1002. \textit{Bottom panel}: SNO V-band photometry of GJ 1002. (*gj1002_data*)

</div>
<div id="div_fig3">

<img src="tmp_2212.07332/./ANALYSIS/RV/RV_DATA_MODEL_0P.png" alt="Fig11" width="100%"/>

**Figure 11. -** Activity-only model. \textit{Top panels}: Combined FWHM time series of ESPRESSO and CARMENES with the best fit model. The right panel shows a zoom to the latest ESPRESSO campaign. \textit{Middle panels}: Combined RV time series of ESPRESSO and CARMENES with the best fit activity-only model. \textit{Bottom panels}: Residuals after the activity-only fit of the RV time series of ESPRESSO and CARMENES. The right panel shows the GLS periodogram of the residuals. The computation of the periodogram uses the offsets and the jitters estimated for model 0-Planets in table \ref{parameters_1}. (*model_rv_0p*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

133  publications in the last 7 days.
	 _build/html/2212.07332.md
	 _build/html/2212.06434.md
	 _build/html/2212.04710.md
	 _build/html/2212.04026.md
	 _build/html/2212.03563.md
	 _build/html/2212.03514.md
	 _build/html/2212.02667.md
	 _build/html/2212.02627.md
	 _build/html/2212.02235.md
	 _build/html/2212.01707.md
	 _build/html/2212.01526.md
	 _build/html/2212.01397.md
	 _build/html/2212.01291.md
	 _build/html/2212.01281.md
	 _build/html/2212.00812.md
	 _build/html/2212.00811.md
	 _build/html/2212.00806.md
	 _build/html/2212.00047.md
	 _build/html/2212.00032.md
	 _build/html/2211.16996.md
	 _build/html/2211.16510.md
	 _build/html/2211.16123.md
	 _build/html/2211.15816.md
	 _build/html/2211.15698.md
	 _build/html/2211.15576.md
	 _build/html/2211.15434.md
	 _build/html/2211.15288.md
	 _build/html/2211.15224.md
	 _build/html/2211.14543.md
	 _build/html/2211.14330.md
	 _build/html/2211.14322.md
	 _build/html/2211.14132.md
	 _build/html/2211.13811.md
	 _build/html/2211.13426.md
	 _build/h

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers